European Central Bank (ECB) API with Python
=====

## Data on Eurozone countries

-----

*September 3, 2017*<br>
*@bd_econ*

ECB API Documentation is [here](https://sdw-wsrest.ecb.europa.eu/web/generator/index.html)

This example requests the interest rate by country on 10-year bonds over the period since 2012.

In [1]:
import requests
import pandas as pd

## Parameters/Settings

List of data sets is [here](https://sdw-wsrest.ecb.europa.eu/service/dataflow)

In [2]:
# List of countries
c_list = ['DE', 
          'FR', 
          'IT', 
          'ES', 
          'PT', 
          'GR', 
          'NL', 
          'BE', 
          'AT', 
          'IE']

param = [('dataflow', 'IRS'),
         ('freq', 'M'),
         ('countries', '+'.join(c_list)),
         ('series', 'L'),
         ('trans type', 'L40'),
         ('maturity cat', 'CI'),
         ('counterpart', '0000'),
         ('currency', 'EUR'),
         ('business coverage', 'N'),
         ('interest rate type', 'Z'),
         ('start', '?startPeriod=2012-01-01')]

param_joined = '.'.join(value for key, value in param[1:-1])

series_key = '{}/{}{}'.format(param[0][1],
                              param_joined,
                              param[-1][1])

## Request data

In [3]:
url = 'https://sdw-wsrest.ecb.europa.eu/service/data/'
# headers used as content negotiation to return data in json format
headers = {'Accept':'application/json'}
r = requests.get('{}{}'.format(url, series_key), headers=headers).json()

## Pandas DataFrame

In [4]:
date_list = r['structure']['dimensions']['observation'][0]['values']
dates = [date['start'][:10] for date in date_list]
    
areas = [v['name'] for v in r['structure']['dimensions']['series'][1]['values']]

df = pd.DataFrame()
for i, area in enumerate(areas):
    s_key = '0:{}:0:0:0:0:0:0:0'.format(i)
    s_list = r['dataSets'][0]['series'][s_key]['observations']
    df[area] = pd.Series([s_list[val][0] for val in sorted(s_list, key=int)])
df.index = dates
df.tail()

,Austria,Belgium,Germany,Spain,France,Greece,Ireland,Italy,Netherlands,Portugal
2018-08-01,0.5973,0.71,0.29,1.396,0.70,4.18,0.86,3.161,0.473,1.819
2018-09-01,0.6654,0.77,0.37,1.459,0.77,4.17,0.93,2.959,0.551,1.881
2018-10-01,0.6875,0.85,0.40,1.595,0.82,4.37,1.01,3.469,0.582,1.957
2018-11-01,0.6201,0.81,0.31,1.587,0.76,4.42,0.98,3.390,0.518,1.924
2018-12-01,0.5198,0.75,0.19,1.423,0.70,4.28,0.91,2.975,0.401,1.725
